# TP : Naïve Bayes et Ensemble appliqués au Titanic

## Récupération des données du cours ANN
Renvoie 2 Dataframe train (avec le champs Survived) et test (sans le champs Survived)

In [1]:
%run ./tp1_prepa_features.ipynb

<span style="color:blue;font-weight:bold">train :</span>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


<span style="color:blue;font-weight:bold">test :</span>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


<span style="color:blue;font-weight:bold">Effet de la classe</span>

...

    Title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


<span style="color:blue;font-weight:bold">train mis en forme :</span>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         891 non-null float64
Fare        891 non-null float64
Embarked    891 non-null int32
IsAlone     891 non-null int64
Title       891 non-null int64
dtypes: float64(2), int32(2), int64(4)
memory usage: 48.8 KB


<span style="color:blue;font-weight:bold">test mis en forme :</span>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null int32
Age         418 non-null float64
Fare        418 non-null float64
Embarked    418 non-null int32
IsAlone     418 non-null int64
Title       418 non-null int64
dtypes: float64(2), int32(2), int64(3)
memory usage: 19.7 KB


##  Préparation des données de Training

In [2]:
from sklearn.model_selection import train_test_split
#Séparation des valeurs de train et label (tous les exemples)
X_alltrain = train.values[:, 1:]
y_alltrain = train.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X_alltrain, y_alltrain, random_state=42)
print('%i X_train, %i X_test, %i y_train,%i y_test'%(
    X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]))
print('%i X_alltrain, %i y_alltrain'%(X_alltrain.shape[0], y_alltrain.shape[0]))
feature_names=train.columns.tolist()[1:]
target_names=["Disparu","Rescapé"]
print('features:',feature_names)
print('target:',target_names)

668 X_train, 223 X_test, 668 y_train,223 y_test
891 X_alltrain, 891 y_alltrain
features: ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'IsAlone', 'Title']
target: ['Disparu', 'Rescapé']


## Fonction Utilitaires

In [3]:
# Fonctions permettant de générer le fichier d'envoi à Kaggle
#
#parametres: Classifiers; Données à calculer ; index)
def generer_resultats(clf,data=test.values,idx=finalfile_index):
    """
    Fonctions permettant de générer le fichier d'envoi à Kaggle.
    On passe un classifier sur lequel on refait le training avec toutes les données de training
    
    Parameters
    ----------
    Classifiers : Classifier utilisé pour la prédiction
    data : Données à calculer. par défaut, les valeurs du dataset "test"
    idx : Index des passagers testés. Stockés dans finalfile_index lors de la lecture des données
    """    
    print(clf.get_params())
    clf.fit(X_alltrain, y_alltrain)
    prediction=clf.predict(data)
    results=pd.DataFrame(prediction.astype(int), index = finalfile_index, columns=['Survived'])
    results.to_csv('resultats%s.csv'%clf.__class__.__name__)
    
#Fonction pour l'affichage 2 D des résultats    
from matplotlib.colors import ListedColormap
def plot_decision_boundary(clf,X,y, axes=[-0, 30, -5, 5], axis_name=['x1','x2'],alpha=0.5, contour=True):
    """
    Fonction pour l'affichage 2 D des résultats   
    
    Parameters
    ----------
    clf : Classifier à afficher
    X : features de Données a afficher
    y : labels de Données a afficher  
    axes : : Tailles des axes (valeur min/max)
    axis_name : Nom des axes sur le graphique
    alpha : Transparence des points
    contour : Afichage du contour
    """     
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bo",label="Disparu", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "ys", label="Rescapé",alpha=alpha)
    plt.axis(axes)
    plt.xlabel(axis_name[0], fontsize=18)
    plt.ylabel(axis_name[1]+ "  ",fontsize=18, rotation=0)    
    plt.legend(loc="lower right", fontsize=14)

## Exercice 1 : Naïve Bayes
En utilisant l'exemple du cours 2, contruisez et testez un modèle bayésien pour prévoir la survie. <br>
Essayez plusieurs valeurs pour max_depth <br>
Envoyer à Kaggle en utilisant la fonction generer_resultats

In [4]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score

gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
y_pred = gnb_clf.predict(X_test)    
print("Niveau de précision : %.2f"%(100*accuracy_score(y_test, y_pred)))


Niveau de précision : 77.13


## Exercice 2 : Modèles d'ensembles
En utilisant les l'exemple 2 du cours 1 ainsi que les résultat du TP1, créer un modèle d'ensemble en testant soft et hard voting.

In [18]:
# Graphes
%matplotlib inline
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

#Eviter les warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
bayes_clf = GaussianNB()

voting_clf = VotingClassifier(estimators=[('lr', log_clf),
                                          ('rf', rnd_clf), 
                                          ('by', bayes_clf)],
                              voting='hard')

dict_scores={}
for clf in (log_clf, rnd_clf, bayes_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)    
    dict_scores[clf.__class__.__name__]=accuracy_score(y_test, y_pred)
    
voting_soft_clf = VotingClassifier(estimators=[('lr', log_clf),
                                          ('rf', rnd_clf),
                                          ('by', bayes_clf)],
                              voting='soft')
voting_soft_clf.fit(X_train, y_train)
y_pred = voting_soft_clf.predict(X_test)
dict_scores['VotingClassifier Soft']=accuracy_score(y_test, y_pred)
dict_scores

generer_resultats(voting_soft_clf)

{'estimators': [('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)), ('by', GaussianNB(priors=None))], 'flatten_transform': None, 'n_jobs': 1, 'voting': 'soft', 'weights': None, 'lr': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random

## Exercice 3 Optionnel : Stacking
Remplacer le voting par un modèle logistique (voir exemple 2 cours 1)
Vous pouvez utiliser la cross validation

In [21]:
from mlxtend.classifier import StackingClassifier

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)

lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, gnb_clf], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([clf1, clf2, gnb_clf, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'Naive Bayes',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))
    
generer_resultats(sclf)

3-fold cross validation:

Accuracy: 0.91 (+/- 0.01) [KNN]
Accuracy: 0.91 (+/- 0.06) [Random Forest]
Accuracy: 0.92 (+/- 0.03) [Naive Bayes]
Accuracy: 0.95 (+/- 0.03) [StackingClassifier]
{'kneighborsclassifier': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform'), 'randomforestclassifier': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False), 'gaussiannb': GaussianNB(priors=None), 'kneighborsclassifier__algorithm': 'auto', 'kneighborsclassifier__leaf_size': 30, 'kneighborsclassifier__metric': 'minkowski', 'kneighborsclassifier__m

## Exercice 4 Optionnel : Ajouter les prédictions tensorflow du cours Neural Network au modèle d'ensemble